In [9]:
import numpy as np

import numpy as np
import pandas as pd
from tabulate import tabulate
from pandas import Series
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display
import re
col_list = ['product_id','product_name','price_mode','price_item','url','cat1','cat2','cat_url','store_name']
products = pd.read_csv("./raw_data.csv", usecols=col_list, )
display(products.head())
print(products.shape)

,product_id,product_name,price_mode,price_item,url,cat1,cat2,cat_url,store_name
0,5034680-EA-000PNS,Nescafe Dolce Gusto Skinny Light Capsules,ea,8.79,/product/5034680_ea_000pns?name=skinny-light-c...,Drinks,Hot Drinks,/category/drinks/hot-drinks,PAK'nSAVE Taupo
1,5239418-EA-000PNS,Red Seal Infusions Turmeric Lemon & Manuka Hon...,ea,5.59,/product/5239418_ea_000pns?name=infusions-turm...,Drinks,Hot Drinks,/category/drinks/hot-drinks,PAK'nSAVE Taupo
2,5002506-EA-000PNS,Bushells Coffee And Chicory Essence,ea,7.89,/product/5002506_ea_000pns?name=coffee-and-chi...,Drinks,Hot Drinks,/category/drinks/hot-drinks,PAK'nSAVE Taupo
3,5266285-EA-000PNS,Robert Harris Hazelnut Strength 3 Medium Infus...,ea,5.00,/product/5266285_ea_000pns?name=hazelnut-stren...,Drinks,Hot Drinks,/category/drinks/hot-drinks,PAK'nSAVE Taupo
4,5267457-EA-000PNS,Red Seal Immunity Green Tea Bags,ea,2.89,/product/5267457_ea_000pns?name=immunity-green...,Drinks,Hot Drinks,/category/drinks/hot-drinks,PAK'nSAVE Taupo


(421963, 9)


In [18]:
# 去除同一分店内相同产品数据
fdNameSets = list(set(products['store_name']))
fdNameSets.sort() # 分店名单
data = pd.DataFrame()
index = 0
for fdName in fdNameSets:
    # 各分店内所有商品条目
    sub_df = products.loc[products['store_name']==fdName]
    sub_df = sub_df.drop_duplicates(subset=['product_name',"price_item"], keep='first')
    data=data.append(sub_df)
    index = index + 1
data = data.drop(columns=["cat_url","product_id"])
data.columns = ['name','price_mode', 'price', 'url', 'cat', 'cat2', 'fdName']
data.to_csv(r'unique_product.csv', index = True, header=True)
display(data.head())

,name,price_mode,price,url,cat,cat2,fdName
369148,Moccona Strong Cappuccino Sachets,ea,8.99,/product/5239221_ea_000pns?name=strong-cappucc...,Drinks,Hot Drinks,PAK'nSAVE Albany
369149,Nescafe Dolce Gusto Americano Capsules,ea,9.69,/product/5243493_ea_000pns?name=americano-caps...,Drinks,Hot Drinks,PAK'nSAVE Albany
369150,Starbucks Caffe Varona Capsules,ea,7.49,/product/5259922_ea_000pns?name=caffe-varona-c...,Drinks,Hot Drinks,PAK'nSAVE Albany
369151,Robert Harris Italian Roast Coffee Capsules,ea,6.79,/product/5232903_ea_000pns?name=italian-roast-...,Drinks,Hot Drinks,PAK'nSAVE Albany
369152,Moccona Latte Sachets,ea,8.99,/product/5239222_ea_000pns?name=latte-sachets,Drinks,Hot Drinks,PAK'nSAVE Albany


In [19]:
# ,name,price,fdName,leibie,cat,images,info,url
new_df = pd.read_csv("unique_product.csv" )
url_prifix = 'https://www.paknsaveonline.co.nz'
new_df['url'] = url_prifix + new_df['url'].astype(str)
new_df['images'] = ''
new_df['info'] = ''
new_df['leibie'] = ''
col_order = ['name','price','fdName','leibie','cat','images','info','url']
new_df = new_df[col_order]
new_df.index = range(0, len(new_df))
new_df.to_csv (r'unique_product_match.csv', index = True, header=True)
new_df.head()


,name,price,fdName,leibie,cat,images,info,url
0,Moccona Strong Cappuccino Sachets,8.99,PAK'nSAVE Albany,,Drinks,,,https://www.paknsaveonline.co.nz/product/52392...
1,Nescafe Dolce Gusto Americano Capsules,9.69,PAK'nSAVE Albany,,Drinks,,,https://www.paknsaveonline.co.nz/product/52434...
2,Starbucks Caffe Varona Capsules,7.49,PAK'nSAVE Albany,,Drinks,,,https://www.paknsaveonline.co.nz/product/52599...
3,Robert Harris Italian Roast Coffee Capsules,6.79,PAK'nSAVE Albany,,Drinks,,,https://www.paknsaveonline.co.nz/product/52329...
4,Moccona Latte Sachets,8.99,PAK'nSAVE Albany,,Drinks,,,https://www.paknsaveonline.co.nz/product/52392...


In [1]:
# 检查缺失值
data.info()
# 没有缺失值

NameError: name 'data' is not defined

In [21]:
# 分店价格均值
data.groupby('fdName')['price'].agg(np.mean)

fdName
PAK'nSAVE Albany          7.192369
PAK'nSAVE Botany          7.390971
PAK'nSAVE Cameron Road    6.552134
PAK'nSAVE Clarence St     7.655076
PAK'nSAVE Clendon         6.876632
PAK'nSAVE Glen Innes      7.189488
PAK'nSAVE Hastings        6.652279
PAK'nSAVE Hawera          6.455417
PAK'nSAVE Kaitaia         6.093355
PAK'nSAVE Kapiti          6.871478
PAK'nSAVE Kilbirnie       6.990433
PAK'nSAVE Lincoln Road    6.434779
PAK'nSAVE Lower Hutt      6.856488
PAK'nSAVE Mangere         6.435577
PAK'nSAVE Manukau         6.877648
PAK'nSAVE Masterton       6.578031
PAK'nSAVE Mill Street     7.446667
PAK'nSAVE Mt Albert       6.212829
PAK'nSAVE Napier City     6.620929
PAK'nSAVE New Plymouth    6.804111
PAK'nSAVE Palmerston N    7.044839
PAK'nSAVE Papakura        7.010913
PAK'nSAVE Papamoa         6.621669
PAK'nSAVE Petone          7.411538
PAK'nSAVE Porirua         6.427311
PAK'nSAVE Pukekohe        6.203114
PAK'nSAVE Royal Oak       7.572514
PAK'nSAVE Silverdale      7.259166
PAK'nSAVE Syl

In [13]:
# 分店第一类别价格均值
data.groupby(['fdName','cat'])['price'].agg(np.mean)

fdName               cat                        
PAK'nSAVE Albany     Baby, Toddler & Kids           10.650276
                     Beer, Cider & Wine             18.084565
                     Chilled, Frozen & Desserts      6.044100
                     Drinks                          5.486773
                     Fresh Foods & Bakery            7.950245
                                                      ...    
PAK'nSAVE Whangarei  Fresh Foods & Bakery           10.015572
                     Kitchen, Dining & Household     6.594220
                     Pantry                          3.410053
                     Personal Care                   8.312237
                     Pets                            7.754369
Name: price, Length: 350, dtype: float64

In [14]:
# 分店第二类别价格均值
data.groupby(['fdName','cat','cat2'])['price'].agg(np.mean)

fdName               cat                         cat2               
PAK'nSAVE Albany     Baby, Toddler & Kids        Baby Care              10.650276
                     Beer, Cider & Wine          Beer & Cider           14.970282
                                                 Wine                   19.475220
                     Chilled, Frozen & Desserts  Cheese                  6.353160
                                                 Dairy & Eggs            4.537308
                                                                          ...    
PAK'nSAVE Whangarei  Pantry                      Spices & Seasonings     3.172381
                                                 World Foods             2.748333
                     Personal Care               Beauty & Grooming       6.821545
                                                 Health & Wellness      10.776949
                     Pets                        Pet Supplies            7.754369
Name: price, Length: 1408, dt

In [15]:
# 分店价格方差
data.groupby('fdName')['price'].agg(np.var)

fdName
PAK'nSAVE Albany          119.241439
PAK'nSAVE Botany          191.068939
PAK'nSAVE Cameron Road     40.910523
PAK'nSAVE Clarence St      60.204635
PAK'nSAVE Clendon          49.497522
PAK'nSAVE Glen Innes       48.665665
PAK'nSAVE Hastings         37.109257
PAK'nSAVE Hawera           36.898729
PAK'nSAVE Kaitaia          37.697156
PAK'nSAVE Kapiti           41.362867
PAK'nSAVE Kilbirnie        47.579126
PAK'nSAVE Lincoln Road     41.992252
PAK'nSAVE Lower Hutt       43.593086
PAK'nSAVE Mangere          44.254012
PAK'nSAVE Manukau          46.297018
PAK'nSAVE Masterton        41.039736
PAK'nSAVE Mill Street      52.413046
PAK'nSAVE Mt Albert        32.511561
PAK'nSAVE Napier City      41.144786
PAK'nSAVE New Plymouth     46.475989
PAK'nSAVE Palmerston N     47.589650
PAK'nSAVE Papakura         56.015452
PAK'nSAVE Papamoa          37.328189
PAK'nSAVE Petone           91.331110
PAK'nSAVE Porirua          36.978726
PAK'nSAVE Pukekohe         37.959872
PAK'nSAVE Royal Oak       143.0

In [16]:
# 分店第一类别价格方差
data.groupby(['fdName','cat'])['price'].agg(np.var)

fdName               cat                        
PAK'nSAVE Albany     Baby, Toddler & Kids            103.865743
                     Beer, Cider & Wine             1060.418530
                     Chilled, Frozen & Desserts       26.149871
                     Drinks                           12.971214
                     Fresh Foods & Bakery             52.247019
                                                       ...     
PAK'nSAVE Whangarei  Fresh Foods & Bakery            121.196460
                     Kitchen, Dining & Household      25.896553
                     Pantry                            4.416726
                     Personal Care                    36.965984
                     Pets                             52.490887
Name: price, Length: 350, dtype: float64

In [17]:
# 分店第二类别价格方差
data.groupby(['fdName','cat','cat2'])['price'].agg(np.var)

fdName               cat                         cat2               
PAK'nSAVE Albany     Baby, Toddler & Kids        Baby Care               103.865743
                     Beer, Cider & Wine          Beer & Cider             81.933463
                                                 Wine                   1492.164194
                     Chilled, Frozen & Desserts  Cheese                    8.503219
                                                 Dairy & Eggs              5.584754
                                                                           ...     
PAK'nSAVE Whangarei  Pantry                      Spices & Seasonings       2.858816
                                                 World Foods               1.982057
                     Personal Care               Beauty & Grooming        20.582687
                                                 Health & Wellness        54.368187
                     Pets                        Pet Supplies             52.490887
Name: p

In [18]:
# 分店第一类别数量
data.groupby('fdName')['cat'].value_counts().head(50)

fdName                  cat                        
PAK'nSAVE Albany        Pantry                         3095
                        Personal Care                  1465
                        Kitchen, Dining & Household    1027
                        Chilled, Frozen & Desserts     1022
                        Fresh Foods & Bakery            899
                        Beer, Cider & Wine              690
                        Drinks                          595
                        Pets                            384
                        Baby, Toddler & Kids            254
PAK'nSAVE Botany        Pantry                         3019
                        Personal Care                  1548
                        Kitchen, Dining & Household    1181
                        Fresh Foods & Bakery           1094
                        Chilled, Frozen & Desserts     1081
                        Beer, Cider & Wine              852
                        Drinks                  

In [19]:
# 分店第二类别数量
data.groupby(['fdName','cat'])['cat2'].value_counts().head(50)

fdName            cat                          cat2                       
PAK'nSAVE Albany  Baby, Toddler & Kids         Baby Care                      254
                  Beer, Cider & Wine           Wine                           477
                                               Beer & Cider                   213
                  Chilled, Frozen & Desserts   Dairy & Eggs                   364
                                               Frozen Foods                   307
                                               Cheese                         212
                                               Desserts                       139
                  Drinks                       Cold Drinks                    346
                                               Hot Drinks                     249
                  Fresh Foods & Bakery         Deli, Salads & Cooked Meats    252
                                               Bakery                         222
                       

In [20]:
# 分店各项统计值
data.groupby('fdName')['price'].describe()

,count,mean,std,min,25%,50%,75%,max
fdName,,,,,,,,
PAK'nSAVE Albany,9431.0,7.192369,10.919773,0.00,2.9900,4.890,8.7900,749.00
PAK'nSAVE Botany,10113.0,7.390971,13.822769,0.25,2.9900,4.790,8.9900,848.88
PAK'nSAVE Cameron Road,8811.0,6.552134,6.396133,0.00,2.8900,4.490,7.9900,119.79
PAK'nSAVE Clarence St,10625.0,7.655076,7.759165,0.25,3.2900,5.000,9.3900,249.79
PAK'nSAVE Clendon,9637.0,6.876632,7.035448,0.65,2.8900,4.490,8.4900,128.90
PAK'nSAVE Glen Innes,10060.0,7.189488,6.976078,0.25,2.9900,4.890,8.9900,128.90
PAK'nSAVE Hastings,10152.0,6.652279,6.091737,0.20,2.9900,4.590,7.9900,92.99
PAK'nSAVE Hawera,10505.0,6.455417,6.074432,0.00,2.8100,4.490,7.9000,92.99
PAK'nSAVE Kaitaia,9996.0,6.093355,6.139801,0.20,2.7900,3.990,6.9900,128.90


In [21]:
# 分店第一类别各项统计值
data.groupby(['fdName','cat'])['price'].describe()

count       mean        std  \
fdName              cat                                                         
PAK'nSAVE Albany    Baby, Toddler & Kids          254.0  10.650276  10.191454   
                    Beer, Cider & Wine            690.0  18.084565  32.564068   
                    Chilled, Frozen & Desserts   1022.0   6.044100   5.113694   
                    Drinks                        595.0   5.486773   3.601557   
                    Fresh Foods & Bakery          899.0   7.950245   7.228210   
...                                                 ...        ...        ...   
PAK'nSAVE Whangarei Fresh Foods & Bakery         1014.0  10.015572  11.008926   
                    Kitchen, Dining & Household  1147.0   6.594220   5.088866   
                    Pantry                       2248.0   3.410053   2.101601   
                    Personal Care                1600.0   8.312238   6.079966   
                    Pets                          531.0   7.754369   7.245060   

                                                  min     25%    50%      75%  \
fdName              cat                                                         
PAK'nSAVE Albany    Baby, Toddler & Kids         1.19   2.490   7.49  15.9900   
                    Beer, Cider & Wine           1.59  10.990  14.79  18.9900   
                    Chilled, Frozen & Desserts   0.49   3.490   4.99   6.9900   
                    Drinks                       0.85   2.990   4.69   6.8900   
                    Fresh Foods & Bakery         0.99   3.790   4.99   9.4900   
...                                               ...     ...    ...      ...   
PAK'nSAVE Whangarei Fresh Foods & Bakery         0.99   3.690   5.49  12.4975   
                    Kitchen, Dining & Household  0.25   3.055   4.89   8.4900   
                    Pantry                       0.69   1.990   2.99   3.9900   
                    Personal Care                0.74   3.990   6.29  10.4900   
                    Pets                         0.95   2.490   5.49  10.7000   

                                                    max  
fdName              cat                                  
PAK'nSAVE Albany    Baby, Toddler & Kids          43.99  
                    Beer, Cider & Wine           749.00  
                    Chilled, Frozen & Desserts    86.99  
                    Drinks                        20.99  
                    Fresh Foods & Bakery          88.99  
...                                                 ...  
PAK'nSAVE Whangarei Fresh Foods & Bakery         128.90  
                    Kitchen, Dining & Household   32.99  
                    Pantry                        19.99  
                    Personal Care                 49.99  
                    Pets                          39.99  

[350 rows x 8 columns]

In [22]:
# 分店第二类别各项统计值
data.groupby(['fdName','cat','cat2'])['price'].describe()

count  \
fdName              cat                        cat2                         
PAK'nSAVE Albany    Baby, Toddler & Kids       Baby Care            254.0   
                    Beer, Cider & Wine         Beer & Cider         213.0   
                                               Wine                 477.0   
                    Chilled, Frozen & Desserts Cheese               212.0   
                                               Dairy & Eggs         364.0   
...                                                                   ...   
PAK'nSAVE Whangarei Pantry                     Spices & Seasonings  168.0   
                                               World Foods           48.0   
                    Personal Care              Beauty & Grooming    997.0   
                                               Health & Wellness    603.0   
                    Pets                       Pet Supplies         531.0   

                                                                         mean  \
fdName              cat                        cat2                             
PAK'nSAVE Albany    Baby, Toddler & Kids       Baby Care            10.650276   
                    Beer, Cider & Wine         Beer & Cider         14.970282   
                                               Wine                 19.475220   
                    Chilled, Frozen & Desserts Cheese                6.353160   
                                               Dairy & Eggs          4.537308   
...                                                                       ...   
PAK'nSAVE Whangarei Pantry                     Spices & Seasonings   3.172381   
                                               World Foods           2.748333   
                    Personal Care              Beauty & Grooming     6.821545   
                                               Health & Wellness    10.776949   
                    Pets                       Pet Supplies          7.754369   

                                                                          std  \
fdName              cat                        cat2                             
PAK'nSAVE Albany    Baby, Toddler & Kids       Baby Care            10.191454   
                    Beer, Cider & Wine         Beer & Cider          9.051711   
                                               Wine                 38.628541   
                    Chilled, Frozen & Desserts Cheese                2.916028   
                                               Dairy & Eggs          2.363208   
...                                                                       ...   
PAK'nSAVE Whangarei Pantry                     Spices & Seasonings   1.690803   
                                               World Foods           1.407855   
                    Personal Care              Beauty & Grooming     4.536815   
                                               Health & Wellness     7.373479   
                    Pets                       Pet Supplies          7.245060   

                                                                     min  \
fdName              cat                        cat2                        
PAK'nSAVE Albany    Baby, Toddler & Kids       Baby Care            1.19   
                    Beer, Cider & Wine         Beer & Cider         1.59   
                                               Wine                 4.79   
                    Chilled, Frozen & Desserts Cheese               2.49   
                                               Dairy & Eggs         0.49   
...                                                                  ...   
PAK'nSAVE Whangarei Pantry                     Spices & Seasonings  0.89   
                                               World Foods          1.19   
                    Personal Care              Beauty & Grooming    0.74   
                                               Health & Wellness    0.99   
                    Pets                      

In [23]:
# 分店一级分类商品均价- group by 分店
df_price_mean = data.groupby(['fdName','cat'])['price'].mean().sort_values(ascending=False).to_frame().reset_index()
display(df_price_mean)

,fdName,cat,price
0,PAK'nSAVE Botany,"Beer, Cider & Wine",19.028498
1,PAK'nSAVE Silverdale,"Beer, Cider & Wine",18.464672
2,PAK'nSAVE Albany,"Beer, Cider & Wine",18.084565
3,PAK'nSAVE Papakura,"Beer, Cider & Wine",17.393761
4,PAK'nSAVE Royal Oak,"Beer, Cider & Wine",16.743737
...,...,...,...
345,PAK'nSAVE Kaitaia,Pantry,3.487050
346,PAK'nSAVE Porirua,Pantry,3.447450
347,PAK'nSAVE Masterton,Pantry,3.424770
348,PAK'nSAVE Whangarei,Pantry,3.410053


In [24]:
# 分店一级分类商品均价- sort by 价格
df_price_mean = data.groupby(['fdName','cat'])['price'].mean().sort_values(ascending=True).to_frame().reset_index()
display(df_price_mean)

,fdName,cat,price
0,PAK'nSAVE Mangere,Pantry,3.361034
1,PAK'nSAVE Whangarei,Pantry,3.410053
2,PAK'nSAVE Masterton,Pantry,3.424770
3,PAK'nSAVE Porirua,Pantry,3.447450
4,PAK'nSAVE Kaitaia,Pantry,3.487050
...,...,...,...
345,PAK'nSAVE Royal Oak,"Beer, Cider & Wine",16.743737
346,PAK'nSAVE Papakura,"Beer, Cider & Wine",17.393761
347,PAK'nSAVE Albany,"Beer, Cider & Wine",18.084565
348,PAK'nSAVE Silverdale,"Beer, Cider & Wine",18.464672


In [25]:
# 分店二级分类商品均价- group by 分店
df_price_mean = data.groupby(['fdName','cat','cat2'])['price'].mean().sort_values(ascending=False).to_frame().reset_index()
display(df_price_mean)

,fdName,cat,cat2,price
0,PAK'nSAVE Clendon,Fresh Foods & Bakery,Bulk & Loose Foods,34.397297
1,PAK'nSAVE Albany,Fresh Foods & Bakery,Bulk & Loose Foods,32.500000
2,PAK'nSAVE Manukau,Fresh Foods & Bakery,Bulk & Loose Foods,31.691867
3,PAK'nSAVE Silverdale,Fresh Foods & Bakery,Bulk & Loose Foods,31.206667
4,PAK'nSAVE Sylvia Park,Fresh Foods & Bakery,Bulk & Loose Foods,30.766667
...,...,...,...,...
1403,PAK'nSAVE Porirua,Pantry,Spices & Seasonings,2.724081
1404,PAK'nSAVE Porirua,Pantry,Canned & Prepared Foods,2.695761
1405,PAK'nSAVE Tamatea,Pantry,"Sauces, Stock & Marinades",2.649453
1406,PAK'nSAVE Whangarei,Pantry,Canned & Prepared Foods,2.612644


In [26]:
# 分店二级分类商品均价- sort by 价格
df_price_mean = data.groupby(['fdName','cat','cat2'])['price'].mean().sort_values(ascending=True).to_frame().reset_index()
display(df_price_mean)

,fdName,cat,cat2,price
0,PAK'nSAVE Whangarei,Pantry,"Sauces, Stock & Marinades",2.559000
1,PAK'nSAVE Whangarei,Pantry,Canned & Prepared Foods,2.612644
2,PAK'nSAVE Tamatea,Pantry,"Sauces, Stock & Marinades",2.649453
3,PAK'nSAVE Porirua,Pantry,Canned & Prepared Foods,2.695761
4,PAK'nSAVE Porirua,Pantry,Spices & Seasonings,2.724081
...,...,...,...,...
1403,PAK'nSAVE Sylvia Park,Fresh Foods & Bakery,Bulk & Loose Foods,30.766667
1404,PAK'nSAVE Silverdale,Fresh Foods & Bakery,Bulk & Loose Foods,31.206667
1405,PAK'nSAVE Manukau,Fresh Foods & Bakery,Bulk & Loose Foods,31.691867
1406,PAK'nSAVE Albany,Fresh Foods & Bakery,Bulk & Loose Foods,32.500000


In [27]:
# 分店商品数统计-降序
df_item_count = data.groupby('fdName')['name'].count().sort_values(ascending =False).to_frame().reset_index()
display(df_item_count)

,fdName,name
0,PAK'nSAVE Lincoln Road,12550
1,PAK'nSAVE Thames,11682
2,PAK'nSAVE Papakura,11630
3,PAK'nSAVE Palmerston N,11130
4,PAK'nSAVE New Plymouth,11098
5,PAK'nSAVE Petone,11086
6,PAK'nSAVE Kapiti,10710
7,PAK'nSAVE Upper Hutt,10638
8,PAK'nSAVE Clarence St,10625
9,PAK'nSAVE Royal Oak,10591


In [28]:
# 分店第一类别商品数统计-降序
data.groupby(['fdName','cat'])['name'].count().sort_values(ascending =False).to_frame().reset_index()

,fdName,cat,name
0,PAK'nSAVE Lincoln Road,Pantry,4295
1,PAK'nSAVE Thames,Pantry,3937
2,PAK'nSAVE Westgate,Pantry,3554
3,PAK'nSAVE New Plymouth,Pantry,3537
4,PAK'nSAVE Hawera,Pantry,3534
...,...,...,...
345,PAK'nSAVE Pukekohe,"Baby, Toddler & Kids",224
346,PAK'nSAVE Whakatane,"Baby, Toddler & Kids",222
347,PAK'nSAVE Lincoln Road,"Beer, Cider & Wine",13
348,PAK'nSAVE Mt Albert,"Beer, Cider & Wine",10


In [29]:
# 分店第二类别商品数统计-降序
data.groupby(['fdName','cat','cat2'])['name'].count().sort_values(ascending =False).to_frame().reset_index()

,fdName,cat,cat2,name
0,PAK'nSAVE Hastings,Personal Care,Beauty & Grooming,1000
1,PAK'nSAVE Hawera,Personal Care,Beauty & Grooming,1000
2,PAK'nSAVE New Plymouth,Personal Care,Beauty & Grooming,1000
3,PAK'nSAVE Lincoln Road,Personal Care,Beauty & Grooming,1000
4,PAK'nSAVE Silverdale,Personal Care,Beauty & Grooming,999
...,...,...,...,...
1403,PAK'nSAVE Wanganui,Fresh Foods & Bakery,Bulk & Loose Foods,2
1404,PAK'nSAVE Mill Street,Fresh Foods & Bakery,Meal Kits,2
1405,PAK'nSAVE Sylvia Park,Fresh Foods & Bakery,Meal Kits,1
1406,PAK'nSAVE Albany,Fresh Foods & Bakery,Bulk & Loose Foods,1


In [30]:
# 商品名称词频
data.name.str.split(expand=True).stack().value_counts().head(50)

&            55929
Pams         28663
Chicken      16297
Chocolate    15334
Cheese       12886
Food         12102
With         11740
Fresh        10993
Cream        10545
Milk         10315
Sauce        10209
Original      9003
Free          8993
Value         8577
Drink         8554
Fruit         8267
Organic       8195
Cat           7864
Rice          7729
Beef          7656
White         7630
In            7377
Natural       7250
Mix           7091
Wattie's      6973
Yoghurt       6927
Coffee        6550
Lemon         6440
The           6189
Soft          6023
Oil           5896
Juice         5877
Extra         5860
Red           5837
For           5749
Premium       5571
Dry           5153
Vanilla       5151
Coconut       5126
Dog           5108
Classic       5082
Apple         5043
Tablets       5007
Black         4936
Bags          4904
Honey         4901
Tomato        4883
Biscuits      4836
Sugar         4817
Tea           4777
dtype: int64

In [ ]:
# 生成最终数据
col_list = ['name','cat','cat2']
data =  data[col_list]
display(data)
data.to_csv (r'product_classification.csv', index = False, header=True)
data = pd.read_csv("./product_classification.csv",usecols=col_list)
display(data)



In [ ]:
data.groupby(['cat'])['name'].count().sort_values(ascending =False).to_frame()

,name,cat,cat2
369148,Moccona Strong Cappuccino Sachets,Drinks,Hot Drinks
369149,Nescafe Dolce Gusto Americano Capsules,Drinks,Hot Drinks
369150,Starbucks Caffe Varona Capsules,Drinks,Hot Drinks
369151,Robert Harris Italian Roast Coffee Capsules,Drinks,Hot Drinks
369152,Moccona Latte Sachets,Drinks,Hot Drinks
...,...,...,...
411216,Yates Nature's Way Fungus Spray,"Kitchen, Dining & Household",Garage & Outdoor
411217,Yates Nature's Way Pyrethum Natural Insect Spray,"Kitchen, Dining & Household",Garage & Outdoor
411218,Selleys All Plastics Glue,"Kitchen, Dining & Household",Garage & Outdoor
411219,Holdfast H/F SCREW DRIVER PHILLIPS 100MM,"Kitchen, Dining & Household",Garage & Outdoor


,name,cat,cat2
0,Moccona Strong Cappuccino Sachets,Drinks,Hot Drinks
1,Nescafe Dolce Gusto Americano Capsules,Drinks,Hot Drinks
2,Starbucks Caffe Varona Capsules,Drinks,Hot Drinks
3,Robert Harris Italian Roast Coffee Capsules,Drinks,Hot Drinks
4,Moccona Latte Sachets,Drinks,Hot Drinks
...,...,...,...
390806,Yates Nature's Way Fungus Spray,"Kitchen, Dining & Household",Garage & Outdoor
390807,Yates Nature's Way Pyrethum Natural Insect Spray,"Kitchen, Dining & Household",Garage & Outdoor
390808,Selleys All Plastics Glue,"Kitchen, Dining & Household",Garage & Outdoor
390809,Holdfast H/F SCREW DRIVER PHILLIPS 100MM,"Kitchen, Dining & Household",Garage & Outdoor


In [33]:
data.groupby(['cat'])['name'].count().sort_values(ascending =False).to_frame()

,name
cat,
Pantry,120907
Personal Care,58565
"Chilled, Frozen & Desserts",44104
Fresh Foods & Bakery,42380
"Kitchen, Dining & Household",41830
Drinks,28486
"Beer, Cider & Wine",25314
Pets,16688
"Baby, Toddler & Kids",12537


In [22]:
# 生成最终数据
col_list = ['name', 'cat']
data = pd.read_csv('../data_cleaning/unique_product_match.csv', encoding='utf_8_sig', usecols=col_list)
data.columns = ['name', 'cat']
display(data)

,name,cat
0,Moccona Strong Cappuccino Sachets,Drinks
1,Nescafe Dolce Gusto Americano Capsules,Drinks
2,Starbucks Caffe Varona Capsules,Drinks
3,Robert Harris Italian Roast Coffee Capsules,Drinks
4,Moccona Latte Sachets,Drinks
...,...,...
390806,Yates Nature's Way Fungus Spray,"Kitchen, Dining & Household"
390807,Yates Nature's Way Pyrethum Natural Insect Spray,"Kitchen, Dining & Household"
390808,Selleys All Plastics Glue,"Kitchen, Dining & Household"
390809,Holdfast H/F SCREW DRIVER PHILLIPS 100MM,"Kitchen, Dining & Household"


In [23]:
data = data.drop_duplicates()
display(data)

,name,cat
0,Moccona Strong Cappuccino Sachets,Drinks
1,Nescafe Dolce Gusto Americano Capsules,Drinks
2,Starbucks Caffe Varona Capsules,Drinks
3,Robert Harris Italian Roast Coffee Capsules,Drinks
4,Moccona Latte Sachets,Drinks
...,...,...
390532,Whangarei District Council Official Rubbish Bags,"Kitchen, Dining & Household"
390802,The Gardener THE GARDENER RAIN METER 160 ML EA,"Kitchen, Dining & Household"
390806,Yates Nature's Way Fungus Spray,"Kitchen, Dining & Household"
390809,Holdfast H/F SCREW DRIVER PHILLIPS 100MM,"Kitchen, Dining & Household"


In [24]:
data.to_csv (r'product_classification.csv', index = False, header=True)

## 将数据csv写成eda输入格式

In [25]:
csv_file = 'product_classification.csv'
txt_file = 'product_classification.txt'
col_list = ['name', 'cat']
data = pd.read_csv(csv_file, usecols=col_list)
display(data)

,name,cat
0,Moccona Strong Cappuccino Sachets,Drinks
1,Nescafe Dolce Gusto Americano Capsules,Drinks
2,Starbucks Caffe Varona Capsules,Drinks
3,Robert Harris Italian Roast Coffee Capsules,Drinks
4,Moccona Latte Sachets,Drinks
...,...,...
35784,Whangarei District Council Official Rubbish Bags,"Kitchen, Dining & Household"
35785,The Gardener THE GARDENER RAIN METER 160 ML EA,"Kitchen, Dining & Household"
35786,Yates Nature's Way Fungus Spray,"Kitchen, Dining & Household"
35787,Holdfast H/F SCREW DRIVER PHILLIPS 100MM,"Kitchen, Dining & Household"


In [26]:
text_list = []
for index, row in data.iterrows():
    text_line = row["cat"]+'\t'+row["name"]
    text_list.append(text_line)

In [58]:
with open(txt_file, "w") as my_output_file:
    [my_output_file.write(row + "\n") for row in text_list]
    my_output_file.close()

## 将EDA输出转化为csv格式

In [49]:
df =  pd.read_csv("eda_data_augmented.txt",sep='\t', header=None)
df.columns = ['cat','name']
df

,cat,name
0,Drinks,moccona strong cappuccino sachets
1,Drinks,moccona sachets cappuccino strong
2,Drinks,moccona warm cappuccino sachets
3,Drinks,moccona strong cappuccino sachets
4,Drinks,moccona strong cappuccino coffee sachets
...,...,...
501041,"Kitchen, Dining & Household",holdfast nooky flat head screw driver
501042,"Kitchen, Dining & Household",holdfast flat head screw number one wood
501043,"Kitchen, Dining & Household",driver flat head screw holdfast
501044,"Kitchen, Dining & Household",holdfast flat head piece of tail driver


In [50]:
col_order = ['name','cat']
df = df[col_order]
df = df.drop_duplicates()
df

,name,cat
0,moccona strong cappuccino sachets,Drinks
1,moccona sachets cappuccino strong,Drinks
2,moccona warm cappuccino sachets,Drinks
4,moccona strong cappuccino coffee sachets,Drinks
5,potent moccona strong cappuccino sachets,Drinks
...,...,...
501039,nooky holdfast flat head screw driver,"Kitchen, Dining & Household"
501041,holdfast nooky flat head screw driver,"Kitchen, Dining & Household"
501042,holdfast flat head screw number one wood,"Kitchen, Dining & Household"
501043,driver flat head screw holdfast,"Kitchen, Dining & Household"


In [51]:
df.to_csv('eda_data_augmented.csv') 